In [38]:
from scipy.io import loadmat
import numpy as np

import sklearn as sk
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [47]:

def plot_data(data,duration):
    sampling_freq = 160.0
    time = np.arange(0.0, duration, 1/sampling_freq)
    plt.plot(time,data)

def segment_data(input_data, seg_time=30):
    # 30 seconds 
    segment_points = seg_time * 160 #sampling freq
    splited_data =np.asarray(np.split(input_data.flatten(), segment_points)).T

    return splited_data

def form_data(input_data,attack_data):
    segment_time = 30 #window = 30seconds
    input_ = segment_data(input_data,segment_time)
    attack_ = segment_data(attack_data,segment_time)

    X = np.concatenate((input_,attack_))
    Y = np.concatenate((np.zeros(input_.shape[0]),np.ones(attack_.shape[0]))) #normal = 0, attack = 1

    return X,Y
        
    

In [48]:

    
    #sampling_freq = 160.0
    
def power_spectrum_plot(data, duration):
    #https://www.adamsmith.haus/python/answers/how-to-plot-a-power-spectrum-in-python
    #Plotting a power spectrum of data will plot how much of the data exists at a range of frequencies.
    #The power spectrum is calculated as the square of the absolute value of the discrete Fourier transform
    #time_stop = 120sec
    sampling_freq = 160.0
    time = np.arange(0.0, duration, 1/160) #(start, stop, step)

    fourier_transform = np.fft.rfft(data)

    abs_fourier_transform = np.abs(fourier_transform)

    power_spectrum = np.square(abs_fourier_transform)

    frequency = np.linspace(0, 160/2, len(power_spectrum))

    plt.plot(frequency, power_spectrum)
        

IndentationError: expected an indented block (2842535591.py, line 8)

In [49]:

    
sampling_freq = 160.0

def filter_band(data, duration):
    #high pass and low pass filter        
    sampling_freq = 160.0
    time = np.arange(0.0, duration, 1/sampling_freq)
    low_freq = 0.5 #0.1 Hz
    high_freq = 2.0 #60 Hz

    filter = signal.firwin(401, [low_freq, high_freq], pass_zero=False,fs=sampling_freq)

    filtered_signal = signal.convolve(data, filter, mode='same')

    plt.plot(time, filtered_signal)
    

In [50]:

from scipy.io import loadmat
import numpy as np

#Read data
input_data = loadmat('Dataset1.mat') #dict_keys(['__header__', '__version__', '__globals__', 'Raw_Data', 'Sampling_Rate'])
attack_data = loadmat('sampleAttack.mat')#dict_keys(['__header__', '__version__', '__globals__', 'attackVectors'])

#loading data
input_data = input_data['Raw_Data']
attack_data = attack_data['attackVectors']

#matrix of 106*3*19200 == > 106 subjects, 3 times of 2 min per subject, 
#160 Hz sampling rate. (19200 = 120 s * 160 hz) 160 samples per second
print("Input data shape: ", input_data.shape)

#matrix of 106*3*19200 == > 6 attack types | 106 subjects | 3 times | 30 sec per subject, 
#160 Hz sampling rate. (4800 = 30 s * 160 hz) 160 samples per second
print("Attack data shape: ", attack_data.shape)

#Combine all data 
X,Y = form_data(input_data,attack_data)

print(X.shape)
print(Y.shape)


#Divide data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
#Pass an int for reproducible output across multiple function calls
    
    
    
    
    

Input data shape:  (106, 3, 19200)
Attack data shape:  (6, 106, 3, 4800)
(3180, 4800)
(3180,)


In [52]:
def delta_band(data):
    
    #https://dsp.stackexchange.com/questions/45345/how-to-correctly-compute-the-eeg-frequency-bands-with-python
    fs = 165  # Sampling rate
    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0 / fs)
    """Delta Band Values"""
    low_freq = 0
    high_freq = 4

    freqs = fft_freq[np.where((fft_freq >= low_freq) &   #np.where is like asking "tell me where in this array, entries satisfy a given condition".
                       (fft_freq <= high_freq))]
    return freqs

In [53]:
def calc_bands_power(x, dt, bands):
    from scipy.signal import welch
    f, psd = welch(x, fs=1. / dt)
    power = {band: np.mean(psd[np.where((f >= lf) & (f <= hf))]) for band, (lf, hf) in bands.items()}
    return power

In [58]:
X.shape

(3180, 4800)

In [66]:
kk = delta_band(X[0])

In [67]:
kk.shape

(117,)

In [73]:
kk = np.array(list(map(delta_band, X)))

In [74]:
kk.shape

(3180, 117)